In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import secrets
print(secrets.token_urlsafe(32))

DLEIRxhHp0anXY0_w-ZkL0C8rhMdlJdX8Xp2CzfIRw0


In [ ]:
import asyncio
import json
import os

import redis.asyncio as redis
from dotenv import load_dotenv


async def subscribe_and_print():
    # Initialize Redis connection
    load_dotenv()
    r = redis.Redis(
        host=os.getenv("REDIS_HOST"), port=os.getenv("REDIS_PORT"), db=0
    ).pubsub()

    # Subscribe to channels
    await r.subscribe("gpu_stats", "cpu_stats", "ssh_stats")

    # Print 10 messages total across all channels
    count = 0
    try:
        while count < 10:
            message = await r.get_message()
            if message and message["type"] == "message":
                data = json.loads(message["data"])
                print(f"Channel: {message['channel']}")
                print(f"Data: {json.dumps(data, indent=2)}")
                print("-" * 50)
                count += 1
    finally:
        await r.unsubscribe()
        await r.aclose()


# Run the async function
await subscribe_and_print()


In [ ]:
import nest_asyncio  # Add this import at the top

nest_asyncio.apply()  # Apply patch to allow nested event loops


import asyncio
import json

import websockets


async def test_websocket():
    async with websockets.connect("ws://localhost:8000/ws/stats/gpu") as websocket:
        while True:
            data = await websocket.recv()
            print(f"Received GPU stats: {json.loads(data)}")


asyncio.run(test_websocket())

In [1]:
# Cell 1: Setup
import asyncio
import json
import os

import aiohttp
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv()

MONITOR_URL = os.getenv("MONITOR_URL").replace("0.0.0.0", "localhost")
MONITOR_API_KEY = os.getenv("MONITOR_API_KEY")
HEADERS = {"X-API-Key": MONITOR_API_KEY}

In [3]:
# Cell 2: Get current settings
async with aiohttp.ClientSession() as session:
    async with session.get(f"{MONITOR_URL}/settings", headers=HEADERS) as response:
        settings = await response.json()
        print("Current Settings:")
        print(json.dumps(settings, indent=2))

Current Settings:
{
  "idle_shutdown_minutes": 30,
  "warning_minutes": 5,
  "check_interval_seconds": 60,
  "dry_run": false
}


In [14]:
# Cell 3: Update settings
new_settings = {
    "idle_shutdown_minutes": 1,
    "warning_minutes": 1
}

async with aiohttp.ClientSession() as session:
    async with session.patch(f"{MONITOR_URL}/settings", headers=HEADERS, json=new_settings) as response:
        updated = await response.json()
        print("Updated Settings:")
        print(json.dumps(updated, indent=2))

Updated Settings:
{
  "idle_shutdown_minutes": 1,
  "warning_minutes": 1,
  "check_interval_seconds": 60,
  "dry_run": false
}


In [7]:
# Cell 4: Configure Slack
slack_settings = {
    "slack_token": os.getenv("SLACK_BOT_TOKEN"),
    "slack_channel": os.getenv("SLACK_CHANNEL")
}

async with aiohttp.ClientSession() as session:
    async with session.put(f"{MONITOR_URL}/slack", headers=HEADERS, json=slack_settings) as response:
        result = await response.json()
        print("Slack Configuration Result:")
        print(json.dumps(result, indent=2))

Slack Configuration Result:
{
  "status": "updated",
  "enabled": true
}


In [22]:
# Cell 5: List all instances
async with aiohttp.ClientSession() as session:
    async with session.get(f"{MONITOR_URL}/instances", headers=HEADERS) as response:
        print(response)
        instances = await response.json()
        print("Registered Instances:")
        print(json.dumps(instances, indent=2))

<ClientResponse(http://localhost:8001/instances) [200 OK]>
<CIMultiDictProxy('Date': 'Fri, 22 Nov 2024 16:36:06 GMT', 'Server': 'uvicorn', 'Content-Length': '172', 'Content-Type': 'application/json')>

Registered Instances:
{
  "test-instance-1": {
    "name": "Test Instance",
    "status": "online",
    "last_heartbeat": "2024-11-22T16:36:00.403315",
    "last_activity": "2024-11-22T16:36:00.403305",
    "current_stats": {}
  }
}


In [27]:
# Cell 6: Get metrics
async with aiohttp.ClientSession() as session:
    async with session.get(f"{MONITOR_URL}/metrics", headers=HEADERS) as response:
        metrics = await response.json()
        print("Current Metrics:")
        print(json.dumps(metrics, indent=2))

Current Metrics:
{
  "total_instances": 1,
  "active_instances": 1,
  "instances_with_gpu_activity": 1,
  "instances_with_users": 1
}


In [26]:
# Cell 7: Get specific instance details
instance_id = os.getenv("INSTANCE_ID")  # from your .env

async with aiohttp.ClientSession() as session:
    async with session.get(f"{MONITOR_URL}/instances/{instance_id}", headers=HEADERS) as response:
        if response.status == 200:
            instance = await response.json()
            print(f"Details for instance {instance_id}:")
            print(json.dumps(instance, indent=2))
        else:
            print(f"Error getting instance (status: {response.status})")
            print(await response.text())

Details for instance test-instance-1:
{
  "name": "Test Instance",
  "status": "online",
  "last_heartbeat": "2024-11-22T16:36:00.403315",
  "last_activity": "2024-11-22T16:36:00.403305",
  "current_stats": {}
}


In [2]:
# Cell 8: Test WebSocket connection
import websockets

instance_id = os.getenv("INSTANCE_ID")
# Use AGENT_URL instead of MONITOR_URL
agent_url = os.getenv("AGENT_URL", "http://localhost:8000")  # Add this to your .env file
ws_url = agent_url.replace("http://", "ws://")
uri = f"{ws_url}/ws/stats/{instance_id}/gpu"

async def test_websocket():
    try:
        async with websockets.connect(uri) as websocket:
            print(f"Connected to WebSocket at {uri}, waiting for messages...")
            for _ in range(3):  # Get 3 messages
                data = await websocket.recv()
                print("\nReceived data:")
                print(json.dumps(json.loads(data), indent=2))
    except Exception as e:
        print(f"WebSocket connection failed: {str(e)}")

await test_websocket()

Connected to WebSocket at ws://localhost:8000/ws/stats/test-instance-1/gpu, waiting for messages...

Received data:
[
  {
    "gpu_id": 0,
    "memory_total": 0,
    "memory_used": 0,
    "memory_free": 0,
    "gpu_utilization": 0,
    "memory_utilization": 0,
    "temperature": 0,
    "timestamp": "2024-11-22T16:49:07.209984"
  }
]

Received data:
[
  {
    "gpu_id": 0,
    "memory_total": 0,
    "memory_used": 0,
    "memory_free": 0,
    "gpu_utilization": 0,
    "memory_utilization": 0,
    "temperature": 0,
    "timestamp": "2024-11-22T16:49:18.232771"
  }
]
WebSocket connection failed: received 1012 (service restart); then sent 1012 (service restart)


In [2]:
import asyncio
import os

import nest_asyncio
from dotenv import load_dotenv
from slack_sdk.errors import SlackApiError
from slack_sdk.web.async_client import AsyncWebClient

nest_asyncio.apply()
load_dotenv()


async def test_slack_message(token: str, channel: str):
    client = AsyncWebClient(token=token)

    # Add debug prints
    print(f"Testing with channel: {channel}")
    print(f"Token starts with: {token[:10]}...")

    try:
        # Test auth first
        auth_test = await client.auth_test()
        print(f"Auth test passed! Connected as: {auth_test['user']}")
        # Test sending a message
        response = await client.chat_postMessage(
            channel=channel,
            text="🔧 Test message from GPU Monitor! 🖥️",
            unfurl_links=False,
        )
        print(f"Message sent successfully! Timestamp: {response['ts']}")

        # Test updating the message
        await asyncio.sleep(2)  # Wait 2 seconds
        await client.chat_update(
            channel=channel,
            ts=response["ts"],
            text="🔄 Updated test message from GPU Monitor! 🖥️",
            unfurl_links=False,
        )
        print("Message updated successfully!")

    except SlackApiError as e:
        print(f"Error: {e.response['error']}")


asyncio.run(
    test_slack_message(os.getenv("SLACK_BOT_TOKEN"), os.getenv("SLACK_CHANNEL"))
)

Testing with channel: bots
Token starts with: xoxb-79564...
Auth test passed! Connected as: powerful_app
Message sent successfully! Timestamp: 1732301896.721719
Error: channel_not_found
